In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/100-most-watched-anime-in-the-world/most_watched_anime_dataset_100_entries.csv


In [2]:
import warnings
warnings.filterwarnings("ignore")

In [3]:
df = pd.read_csv("/kaggle/input/100-most-watched-anime-in-the-world/most_watched_anime_dataset_100_entries.csv")

I want to focus primarily on ratings

In [4]:
df.isnull().sum()

Anime Name                        10
Most Watched in Country           10
Ratings                           10
Number of Episodes                10
Animation Studio Name             10
Budget (in Million USD)           10
Release Year                      10
Genre                             10
Duration per Episode (minutes)    10
dtype: int64

In [5]:
df.dropna(inplace=True)

In [6]:
!pip install altair -- upgrade
!pip install vega_datasets

ERROR: Could not find a version that satisfies the requirement upgrade (from versions: none)
ERROR: No matching distribution found for upgrade


In [7]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px
import altair as alt

In [8]:
fig1 = px.density_contour(df, x="Ratings", y="Budget (in Million USD)", 
                         title="KDE Density Plot: Ratings vs Budget")

fig1.update_layout(
    xaxis_title="Ratings",
    yaxis_title="Budget (Million USD)",
    template="plotly_dark"
)

fig1.show()
#fig1.write_image("plotly_chart.png")

In [9]:
fig2 = px.scatter_matrix(df, 
                        dimensions=['Ratings','Number of Episodes', 'Budget (in Million USD)', 'Release Year'],
                        color='Ratings',  
                        title="📊 Pairplot of Anime Dataset",
                        template="plotly_dark")

fig2.update_layout(
    font=dict(size=12),
    width=900, height=900
)

fig2.show()
#fig.write_image("plotly_chart.png")

In [10]:
df1 = df.loc[df.groupby("Anime Name")["Budget (in Million USD)"].idxmax()]
df1 = df1[df1["Budget (in Million USD)"] > 50].sort_values(by="Budget (in Million USD)", ascending=False).head(5)

fig3 = px.bar(df1, x="Anime Name", y="Budget (in Million USD)", 
             title="Most High-Budget Anime",
             text="Budget (in Million USD)",  
             color="Anime Name",  
             color_discrete_sequence=px.colors.qualitative.Set1,
             hover_data={"Ratings": True, "Anime Name": False, "Budget (in Million USD)": False}  # Show only Ratings
            )  

fig3.update_traces(texttemplate='%{text:.1f}M', textposition="outside")

fig3.update_layout(
    xaxis_title="Anime Name",
    yaxis_title="Budget (Million USD)",
    font=dict(size=13),
    template="plotly_dark", 
    xaxis_tickangle=-15  
)

fig3.show()
#fig.write_image("plotly_chart.png")

Pie Chart

In [11]:
fig4 = px.pie(df, names="Genre", 
             title="Genre Distribution of Anime",
             color_discrete_sequence=px.colors.qualitative.Set2, 
             hole=0.3)  

fig4.update_traces(textinfo="percent+label", 
                  pull=[0.05 if i == df['Genre'].value_counts().idxmax() else 0 for i in df['Genre'].unique()],  # Pops out the most common genre
                  marker=dict(line=dict(color='black', width=1)))  

fig4.update_layout(
    showlegend=True,
    font=dict(size=13),
    template="plotly_dark"  
)

fig4.show()
#fig.write_image("plotly_chart.png")

In [12]:
fig5 = px.box(df, x="Genre", y="Ratings", 
             title="Rating Distribution by Genre",
             color="Genre",
             color_discrete_sequence=px.colors.qualitative.Set2)

fig5.update_layout(
    xaxis_title="Genre",
    yaxis_title="Ratings",
    font=dict(size=13),
    template="plotly_dark",
    xaxis_tickangle=-15  
)

fig5.show()
#fig.write_image("box_plot.png")

In [13]:
fig6 = px.scatter(df, x="Budget (in Million USD)", y="Ratings", 
                 color="Genre",  
                 hover_data=["Anime Name", "Animation Studio Name"],  
                 template="plotly_dark",  
                 opacity=0.8)  

fig6.update_layout(
    title="Budget vs Ratings of Anime",
    xaxis_title="Budget (Million USD)",
    yaxis_title="Ratings",
    font=dict(size=13),
    legend_title="Genre"
)

fig6.show()
#fig.write_image("plotly_chart.png")

In [14]:
import plotly.express as px

fig7 = px.choropleth(df, 
                    locations="Most Watched in Country",  
                    locationmode="country names",  
                    color="Anime Name",  # Color based on the most popular anime
                    hover_name="Most Watched in Country",  
                    title="Most Popular Anime by Country",  
                    color_discrete_sequence=px.colors.qualitative.Set3)  # Use distinct colors

fig7.update_layout(
    geo=dict(showcoastlines=True, showland=True, landcolor="lightgray"),  
    template="plotly_dark",  
    font=dict(size=12, color="white"),  
    title_font_size=20
)

fig7.show()
#fig.write_image("most_popular_anime_by_country.png")